# Advent of Code 2017

[see here](https://adventofcode.com/2017/)

## Preparation

import of some useful libs and definition of some common utility functions

In [41]:
# Python 3.x
import re
import numpy as np
import math
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    
    filename = 'input/input{}.txt'.format(day)
    try:
        with open(filename, 'r') as f:
            text = f.read()
        return text
    except FileNotFoundError:
        url = 'http://adventofcode.com/2017/day/{}/input'.format(day)
        print('input file not found. opening browser...')
        print('please save the file as "input<#day>.txt in your input folder.')
        import webbrowser
        webbrowser.open(url)

cat = ''.join
first = lambda x: list(x)[0]

def shift(it, n):
    return it[n:] + it[:n]

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

## Day 1

In [11]:
captcha = Input(1).strip()
matching = (int(x) for x,y in zip(captcha,shift(captcha,1)) if x==y)
print('Day 1 part 1 result: ' +str(sum(matching)))

Day 1 part 1 result: 995


In [12]:
matching = (int(x) for x,y in zip(captcha,shift(captcha,len(captcha)//2)) if x==y)
print('Day 1 part 2 result: ' +str(sum(matching)))

Day 1 part 2 result: 1130


## Day 2

In [19]:
ss = Input(2).strip()
numbers = [[int(x) for x in line.split('\t')] for line in ss.split('\n')]
checksum = sum(max(line)-min(line) for line in numbers)
print('Checksum is '+str(checksum))

Checksum is 48357


In [20]:
result = sum([x/y for x,y in combinations(line)][0] for line in numbers)

TypeError: Required argument 'r' (pos 2) not found

In [42]:
sum(first(int(max(x)/min(x)) for x in combinations(line,2) if max(x)%min(x) == 0) for line in numbers)

351.0